# Setup environment & mount google drive
import/install packages etc

In [ ]:
!python --version
!pip install --upgrade numpy==1.24
!pip install scikit-multiflow

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
np.float = float    #module 'numpy' has no attribute 'float'
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    #module 'numpy' has no attribute 'bool'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from skmultiflow.meta import OnlineBoostingClassifier, AdaptiveRandomForestClassifier
from skmultiflow.lazy import KNNADWINClassifier
from skmultiflow.data import FileStream
from skmultiflow.utils.data_structures import ConfusionMatrix
from skmultiflow.drift_detection.adwin import ADWIN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

In [ ]:
#Importing csv file
df = pd.read_csv('/content/drive/MyDrive/code/malware-11-families-data-10000.csv') #10000 samples
#df = pd.read_csv('/content/drive/MyDrive/code/malware-4-families-data-8609.csv') #8609 samples
#df = pd.read_csv('/content/drive/MyDrive/code/malware-20-families-data-23777.csv') #23777 samples
df
#df.head()
#df.describe()

In [ ]:
print(df.isnull().any())
print("Number of columns with missing values: ", df.isnull().any().sum())

# Testing for Concept Drift using ADWIN

In [ ]:
data_stream = df["target"]

adwin = ADWIN()

previous_variance = 0
for i in range(10000):
    adwin.add_element(data_stream[i])
    if adwin.detected_change():
        print("Change detected in value {}, at index {}".format(data_stream[i], i))
        print("Current variance: {}. Previous variance {}".format(adwin.variance, previous_variance))
    previous_variance = adwin.variance

# ADAPTIVE MODELS:

*   Adaptive Random Forest
*   KNNADWIN
*   Online Boosting



In [ ]:
target_column = df['target']
print(target_column)

In [ ]:
#shuffle rows
df = df.sample(frac = 1)
df.to_csv('file-1.csv', index=False)
df

# Adaptive Random Forest

In [ ]:
stream = FileStream('file-1.csv')
arf = AdaptiveRandomForestClassifier()

n_samples = 0
correct_cnt = 0
max_samples = 10000

#precision = TP/(TP+FP)
#recall = TP/(TP+FN)
#f1 = (2*precision*recall)/(precision+recall)

truePositive = [0,0,0,0,0,0,0,0,0,0,0]
predictedTotalPositiveValues = [0,0,0,0,0,0,0,0,0,0,0]
instancesCountOfTarget = [0,0,0,0,0,0,0,0,0,0,0]

cm1 = ConfusionMatrix(11,np.float64)

while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = arf.predict(X)

    if y[0] == y_pred[0]:
        correct_cnt += 1
        truePositive[y_pred[0]] += 1 #incrementing correct predictions of target class y_pred[0]
    predictedTotalPositiveValues[y_pred[0]] += 1 #increment total number of y_pred[0] predictions
    instancesCountOfTarget[y[0]] += 1 #value to represent all instances of target

    arf.partial_fit(X, y)
    n_samples += 1
    cm1.update(y, y_pred)
    print(n_samples)

print('Adaptive Random Forest:')
print('{} samples analyzed.'.format(n_samples))
print('Accuracy: {}'.format(correct_cnt / n_samples))

print("truePositive: ", truePositive) #printing the array
print("predictedTotalPositiveValues: ", predictedTotalPositiveValues) #printing the array
print("instancesCountOfTarget: ", instancesCountOfTarget) #printing the array

print('Family#        Precision        Recall        F1')
for x in range(11):
  precision = truePositive[x]/predictedTotalPositiveValues[x]
  recall = truePositive[x]/instancesCountOfTarget[x]
  f1 = (2*precision*recall)/(precision+recall)
  print('{}:        {}        {}        {}'.format(x, precision, recall, f1))

In [ ]:
cm1.matrix.view()

In [ ]:
ax = sns.heatmap(cm1.matrix.view(), annot=True, cmap="YlGnBu", fmt='.3g', xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])
ax.set_title('Confusion Matrix of Adaptive Random Forest Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

# KNNADWIN

In [ ]:
stream1 = FileStream('file-1.csv')
knn_adwin = KNNADWINClassifier(n_neighbors=7, leaf_size=40, max_window_size=1000)

n_samples = 0
correct_cnt = 0
max_samples = 10000

truePositive = [0,0,0,0,0,0,0,0,0,0,0]
predictedTotalPositiveValues = [0,0,0,0,0,0,0,0,0,0,0]
instancesCountOfTarget = [0,0,0,0,0,0,0,0,0,0,0]

cm2 = ConfusionMatrix(11,np.float64)

while n_samples < max_samples and stream1.has_more_samples():
    X, y = stream1.next_sample()
    y_pred = knn_adwin.predict(X)

    if y[0] == y_pred[0]:
        correct_cnt += 1
        truePositive[y_pred[0]] += 1 #incrementing correct predictions of target class y_pred[0]
    predictedTotalPositiveValues[y_pred[0]] += 1 #increment total number of y_pred[0] predictions
    instancesCountOfTarget[y[0]] += 1 #value to represent all instances of target

    knn_adwin.partial_fit(X, y)
    n_samples += 1
    cm2.update(y, y_pred)
    print(n_samples)

print('KNNADWIN:')
print('{} samples analyzed.'.format(n_samples))
print('Accuracy: {}'.format(correct_cnt / n_samples))

print("truePositive: ", truePositive) #printing the array
print("predictedTotalPositiveValues: ", predictedTotalPositiveValues) #printing the array
print("instancesCountOfTarget: ", instancesCountOfTarget) #printing the array

print('Family#        Precision        Recall        F1')
for x in range(11):
  precision = truePositive[x]/predictedTotalPositiveValues[x]
  recall = truePositive[x]/instancesCountOfTarget[x]
  f1 = (2*precision*recall)/(precision+recall)
  print('{}:        {}        {}        {}'.format(x, precision, recall, f1))

In [ ]:
cm2.matrix.view()

In [ ]:
ax = sns.heatmap(cm2.matrix.view(), annot=True, cmap="YlGnBu", fmt='.3g', xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])
ax.set_title('Confusion Matrix of KNNADWIN Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

# Online Boosting

In [ ]:
stream1 = FileStream('file-1.csv')
online_boosting = OnlineBoostingClassifier()

n_samples = 0
correct_cnt = 0
max_samples = 10000

#precision = TP/(TP+FP)
#recall = TP/(TP+FN)
#f1 = (2*precision*recall)/(precision+recall)

truePositive = [0,0,0,0,0,0,0,0,0,0,0]
predictedTotalPositiveValues = [0,0,0,0,0,0,0,0,0,0,0]
instancesCountOfTarget = [0,0,0,0,0,0,0,0,0,0,0]

cm3 = ConfusionMatrix(11,np.float64)

while n_samples < max_samples and stream1.has_more_samples():
    X, y = stream1.next_sample()
    y_pred = online_boosting.predict(X)

    if y[0] == y_pred[0]:
        correct_cnt += 1
        truePositive[y_pred[0]] += 1 #incrementing correct predictions of target class y_pred[0]
    predictedTotalPositiveValues[y_pred[0]] += 1 #increment total number of y_pred[0] predictions
    instancesCountOfTarget[y[0]] += 1 #value to represent all instances of target

    online_boosting.partial_fit(X, y, classes=np.unique(target_column))
    n_samples += 1
    cm3.update(y, y_pred)
    print(n_samples)

print('Online Boosting:')
print('{} samples analyzed.'.format(n_samples))
print('Accuracy: {}'.format(correct_cnt / n_samples))

print("truePositive: ", truePositive) #printing the array
print("predictedTotalPositiveValues: ", predictedTotalPositiveValues) #printing the array
print("instancesCountOfTarget: ", instancesCountOfTarget) #printing the array

print('Family#        Precision        Recall        F1')
for x in range(11):
  precision = truePositive[x]/predictedTotalPositiveValues[x]
  recall = truePositive[x]/instancesCountOfTarget[x]
  f1 = (2*precision*recall)/(precision+recall)
  print('{}:        {}        {}        {}'.format(x, precision, recall, f1))

In [ ]:
cm3.matrix.view()

In [ ]:
ax = sns.heatmap(cm3.matrix.view(), annot=True, cmap="YlGnBu",fmt='.3g',xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Online Boosting Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

# NON-ADAPTIVE MODELS:

*   Random Forest
*   K-Nearest Neighbors
*   Support Vector Machine
*   Word2Vec (CBOW & SKIPGRAM)
*   AdaBoost

# Random Forest

In [ ]:
#split into train, validation and test dataset
x = df.drop('target', axis = 1).copy(deep = True)
y = df['target'].copy(deep = True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=40, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1,random_state=40,stratify=y_train)


In [ ]:
#RandomForest
RFC = RandomForestClassifier(n_estimators=200, min_samples_split=5, random_state=40).fit(x_train, y_train)

ypred = RFC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Random Forest - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

# 11 families
target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']

# 20 families
#target_names = ['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot']
# 4 families
#target_names = ['Obfuscator','Winwebsec','Zbot', 'Vobfus']

print(classification_report(y_val, ypred, target_names=target_names))


ypred = RFC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

# 20 families
#ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot'])
# 4 families
#ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Obfuscator','Winwebsec','Zbot','Vobfus'],yticklabels=['Obfuscator','Winwebsec','Zbot','Vobfus'])

ax.set_title('Confusion Matrix of Random Forest Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Random Forest - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

# 11 families
target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']

# 20 families
#target_names = ['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot']
# 4 families
#target_names = ['Obfuscator','Winwebsec','Zbot', 'Vobfus']

print(classification_report(y_test, ypred, target_names=target_names))

#K-Nearest Neighbors Classifier

In [ ]:
#K-Nearest Neighbors
KNC = KNeighborsClassifier().fit(x_train, y_train)

ypred = KNC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = KNC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of K Neighbors Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

# AdaBoost

In [ ]:
n_estimators = 300

abc = AdaBoostClassifier(
    n_estimators=n_estimators,
    algorithm="SAMME",
    random_state=42,
).fit(x_train, y_train)

ypred = abc.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for AdaBoost - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))


ypred = abc.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of AdaBoost Classifier')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for AdaBoost - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

#Support Vector Machine

In [ ]:
#SVM
SVM = svm.SVC(kernel='linear',C=1.0).fit(x_train, y_train)

ypred = SVM.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Support Vector Machine - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = SVM.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Support Vector Machine')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Support Vector Machine - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

# Word2Vec

## CBOW

In [ ]:
df_enc = pd.read_csv('/content/drive/MyDrive/code/malware-11-families-data-10000.csv')
df_enc1 = pd.read_csv('/content/drive/MyDrive/code/malware-11-families-data-10000.csv')

first_column = df_enc.pop('target')
df_enc.insert(0, 'target', first_column)

first_column = df_enc1.pop('target')
df_enc1.insert(0, 'target', first_column)

df_enc
#make sure target column is at beginning

In [ ]:
columns_without_target=df_enc.columns[1:]

df_enc['op_sequence']=df_enc[columns_without_target].apply(lambda x: [str(opcode) for opcode in x], axis=1)
df_enc['op_sequence']

In [ ]:
df_enc=df_enc.drop(columns=columns_without_target)
df_enc

In [ ]:
#CBOW:
cbow = Word2Vec(sentences=df_enc['op_sequence'],vector_size=425,window=5,min_count=1,workers=4,sg=0) #sg=0 means not skip gram

def file_embeddings(opcodes):
	vectors = [cbow.wv[str(opcode)] for opcode in opcodes if str(opcode) in cbow.wv]
	return sum(vectors) / len(vectors) if vectors else [0] * cbow.vector_size

df_enc['op_embeddings'] = df_enc['op_sequence'].apply(file_embeddings)
df_enc

In [ ]:
x = list(df_enc['op_embeddings'])
y = df_enc['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=40, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1,random_state=40,stratify=y_train)

### RF with CBOW

In [ ]:
#RandomForest with CBOW
RFC = RandomForestClassifier(n_estimators=200, min_samples_split=5, random_state=40).fit(x_train, y_train)

ypred = RFC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Random Forest with CBOW - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = RFC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Random Forest Classifier with CBOW')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Random Forest with CBOW - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

### KNN with CBOW

In [ ]:
#KNeighbors with CBOW
KNC = KNeighborsClassifier().fit(x_train, y_train) #n_neighbors default = 5

ypred = KNC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours with CBOW - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = KNC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of K Neighbors Classifier with CBOW')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours with CBOW - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

### SVM with CBOW

In [ ]:
#SVM with CBOW
SVM = svm.SVC(kernel='linear',C=1.0).fit(x_train, y_train)

ypred = SVM.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Support Vector Machine with CBOW - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = SVM.predict(x_test)
#20 families
#ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Delf','Diplugem','Enterak.A','Expiro.BK','FakeRean','Injector','Lamechi.B','Obfuscator','OnLineGames','Small','Startpage','Systex.A','Toga!rfn','VBInject','Vobfus','Vundo','Winwebsec','Zbot'])

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Support Vector Machine with CBOW')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Support Vector Machine with CBOW - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

## Skip-Gram

In [ ]:
columns_without_target=df_enc1.columns[1:]

df_enc1['op_sequence']=df_enc1[columns_without_target].apply(lambda x: [str(opcode) for opcode in x], axis=1)
df_enc1['op_sequence']

In [ ]:
df_enc1=df_enc1.drop(columns=columns_without_target)
df_enc1

In [ ]:
#SKIPGRAM:
cbow = Word2Vec(sentences=df_enc1['op_sequence'],vector_size=425,window=5,min_count=1,workers=4) #DEFAULT IS SKIPGRAM

def file_embeddings(opcodes):
	vectors = [cbow.wv[str(opcode)] for opcode in opcodes if str(opcode) in cbow.wv]
	return sum(vectors) / len(vectors) if vectors else [0] * cbow.vector_size

df_enc1['op_embeddings'] = df_enc1['op_sequence'].apply(file_embeddings)
df_enc1

In [ ]:
x = list(df_enc1['op_embeddings'])
y = df_enc1['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=40, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1,random_state=40,stratify=y_train)

### RF with SKIPGRAM

In [ ]:
#RandomForest with SKIPGRAM
RFC = RandomForestClassifier(n_estimators=200, min_samples_split=5, random_state=40).fit(x_train, y_train)

ypred = RFC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Random Forest with SKIPGRAM - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = RFC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Random Forest Classifier with SKIPGRAM')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Random Forest with SKIPGRAM - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

### KNN with SKIPGRAM

In [ ]:
#KNeighbors with SKIPGRAM
KNC = KNeighborsClassifier().fit(x_train, y_train) #n_neighbors default = 5

ypred = RFC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours with SKIPGRAM - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = KNC.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of K Neighbors Classifier with SKIPGRAM')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for K-Nearest Neighbours with SKIPGRAM - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))

### SVM with SKIPGRAM

In [ ]:
#SVM with SKIPGRAM
SVM = svm.SVC(kernel='linear',C=1.0).fit(x_train, y_train)

ypred = RFC.predict(x_val)
acc = accuracy_score(y_val, ypred)
pre = precision_score(y_val, ypred, average='macro')
rec = recall_score(y_val, ypred, average='macro')
f1 = f1_score(y_val, ypred, average='macro')
print("\nMetrics for Support Vector Machine with SKIPGRAM - Validation Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_val, ypred, target_names=target_names))

ypred = SVM.predict(x_test)

#11 families
ax = sns.heatmap(confusion_matrix(y_test,ypred), annot=True, fmt="d",cmap="YlGnBu",xticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'],yticklabels=['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot'])

ax.set_title('Confusion Matrix of Support Vector Machine with SKIPGRAM')
ax.set_xlabel('Predict')
ax.set_ylabel('Actual')

acc = accuracy_score(y_test, ypred)
pre = precision_score(y_test, ypred, average='macro')
rec = recall_score(y_test, ypred, average='macro')
f1 = f1_score(y_test, ypred, average='macro')
print("\nMetrics for Support Vector Machine with SKIPGRAM - Test Data")
print("Accuracy: {0:.5f}".format(acc))
print("Precision: {0:.5f}".format(pre))
print("Recall: {0:.5f}".format(rec))
print("F1 Score: {0:.5f}".format(f1))

target_names = ['Allaple.A','Beebone','Diplugem','Obfuscator','OnLineGames','Startpage','Systex.A','Vobfus','Vundo','Winwebsec','Zbot']
print(classification_report(y_test, ypred, target_names=target_names))